In [1]:
!pip install -q datasets transformers[torch]
!pip install accelerate==0.27.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.29.3
    Uninstalling accelerate-0.29.3:
      Successfully uninstalled accelerate-0.29.3


In [2]:
# -------------------
# Montar Google Drive
# -------------------

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/HomoMex_2024/public_data_train_phase')

Mounted at /content/drive


In [3]:
PRETRAINED_MODEL = 'hiramochoavea/homomex24-t2-beto-85-15'

DATASET_FILE = 'new_train_track2_75.csv'
TEST_FILE = 'new_test_track2_15.csv'
VALIDATION_FILE = 'new_validation_track2_10.csv'

In [4]:
# ------------------------------------------------------------------------------
# Cargar dataset original. Por lo pronto, únicamente tenemos el conjunto train.
# ------------------------------------------------------------------------------

from datasets import load_dataset

data_files =  {
                "train": DATASET_FILE,
                "test": TEST_FILE,
                "validation": VALIDATION_FILE
              }

dict_homomex = load_dataset("csv", data_files=data_files)

dict_homomex = dict_homomex.rename_columns({"content": "text"})

dict_homomex

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR'],
        num_rows: 804
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR'],
        num_rows: 107
    })
})

In [5]:
dict_homomex['test']

Dataset({
    features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR'],
    num_rows: 160
})

In [6]:
from datasets import Dataset

REDUCE_DATA = False  # True for running this notebook during the class (90 minutes). We will only use a small sample of the dataset

def get_smaller_sample(data: Dataset, portion=0.05):
    size = round(data.num_rows*portion)
    # print(size)
    data = data.shuffle(seed = 42).select(range(size))
    return data

if REDUCE_DATA:
    # we only take a small sample from each split
    for split in dict_homomex.keys():
        print(split)
        if split == 'test':
            dict_homomex[split]=get_smaller_sample(dict_homomex[split], 0.01)
        else:
            dict_homomex[split]=get_smaller_sample(dict_homomex[split])

dict_homomex

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR'],
        num_rows: 804
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR'],
        num_rows: 107
    })
})

In [7]:
TARGET_LABELS=list(dict_homomex['train'].features)[2:]
print(TARGET_LABELS)

['L', 'G', 'B', 'T', 'O', 'NR']


In [8]:
import torch

def join_labels(example):
    labels = []
    for name_label in TARGET_LABELS:
        labels.append(example[name_label])

    example['label']=torch.Tensor(labels)

    return example


new_dic_homomex = dict_homomex.map(join_labels)
new_dic_homomex

Map:   0%|          | 0/804 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR', 'label'],
        num_rows: 804
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR', 'label'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR', 'label'],
        num_rows: 107
    })
})

In [9]:
num_nonzero = 0
for i in range(new_dic_homomex['train'].num_rows):
    if 1 in new_dic_homomex['train'][i]['label']:
        num_nonzero += 1

print(num_nonzero)

804


In [10]:
from transformers import BertTokenizer, BertTokenizerFast, BertModel, BertConfig
model_name= PRETRAINED_MODEL
tokenizer = BertTokenizerFast.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/735k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [11]:
MAX_LENGTH= max([len(tokenizer(text).input_ids) for text in new_dic_homomex['train']['text']])
print("Tamaño máximo de tokens:", MAX_LENGTH)

Tamaño máximo de tokens: 125


In [12]:
def tokenize(example):
    # postpone the padding
    return tokenizer(example['text'], truncation=True, max_length=MAX_LENGTH, padding='max_length', return_tensors='pt')

In [13]:
encoded_dataset = new_dic_homomex.map(tokenize, batched = True)
encoded_dataset

Map:   0%|          | 0/804 [00:00<?, ? examples/s]

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/107 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 804
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'L', 'G', 'B', 'T', 'O', 'NR', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 107
    })
})

In [14]:
encoded_dataset  = encoded_dataset.remove_columns(dict_homomex['train'].column_names)
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 804
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 160
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 107
    })
})

In [15]:
#print(type(encoded_dataset['train'][0]['label']))

In [16]:
id2label = {idx:label for idx, label in enumerate(TARGET_LABELS)}
label2id = {label:idx for idx, label in enumerate(TARGET_LABELS)}
TARGET_LABELS, id2label, label2id

(['L', 'G', 'B', 'T', 'O', 'NR'],
 {0: 'L', 1: 'G', 2: 'B', 3: 'T', 4: 'O', 5: 'NR'},
 {'L': 0, 'G': 1, 'B': 2, 'T': 3, 'O': 4, 'NR': 5})

In [18]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           problem_type="multi_label_classification",
                                                           num_labels=len(TARGET_LABELS),
                                                           id2label=id2label,
                                                           label2id=label2id,
                                                           ignore_mismatched_sizes=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hiramochoavea/homomex24-t2-beto-85-15 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
from transformers import TrainingArguments, Trainer
batch_size = 32

args = TrainingArguments(
    output_dir='./outputs',
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model= "f1"
)

In [20]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import numpy as np

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions,
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds,
        labels=p.label_ids)
    return result

In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.169568,0.828194,0.881574,0.785047
2,No log,0.143006,0.871111,0.904117,0.831776
3,No log,0.149410,0.871111,0.904117,0.813084
4,No log,0.143904,0.881057,0.912520,0.831776
5,No log,0.144281,0.876106,0.908318,0.822430


TrainOutput(global_step=130, training_loss=0.09181810525747446, metrics={'train_runtime': 132.6689, 'train_samples_per_second': 30.301, 'train_steps_per_second': 0.98, 'total_flos': 258238386090000.0, 'train_loss': 0.09181810525747446, 'epoch': 5.0})

In [24]:
trainer.evaluate()

{'eval_loss': 0.14390435814857483,
 'eval_f1': 0.881057268722467,
 'eval_roc_auc': 0.9125198836704854,
 'eval_accuracy': 0.8317757009345794,
 'eval_runtime': 0.8085,
 'eval_samples_per_second': 132.337,
 'eval_steps_per_second': 4.947,
 'epoch': 5.0}

In [25]:
# def get_prediction(text):
#     # prepare our text into tokenized sequence
#     inputs = tokenizer(text, padding="max_length", truncation=True, return_tensors="pt").to("cuda")
#     # perform inference to our model
#     outputs = model(**inputs)

#     return outputs

def get_prediction(text):
  encoding = tokenizer(text, return_tensors="pt")
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)

  logits = outputs.logits
  logits.shape

  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(logits.squeeze().cpu())
  predictions = np.zeros(probs.shape)
  predictions[np.where(probs >= 0.5)] = 1

  # turn predicted id's into actual label names
  #predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
  #print(predicted_labels)

  return predictions

In [26]:
import torch
torch.cuda.empty_cache()

In [27]:
new_dic_homomex['test'][0]

{'Unnamed: 0': 367,
 'text': '@Adan_Coromoto @NicolasMaduro Par de maricones!!!',
 'L': 0,
 'G': 1,
 'B': 0,
 'T': 0,
 'O': 0,
 'NR': 0,
 'label': [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]}

In [28]:
y_pred = [get_prediction(text) for text in new_dic_homomex['test']['text']]
#y_pred
#y_true = dict_homomex['test']['label']
#y_true

In [29]:
def encode_values(data):
    return [data['L'], data['G'], data['B'], data['T'], data['O'], data['NR']]

y_true = [example['label'] for example in new_dic_homomex['test']]

In [30]:
y_true[:4]

[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [1.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 1.0, 0.0, 0.0, 0.0, 0.0]]

In [31]:
get_prediction("Es de jotitas muy corrientitas hablar y cortar palabras como obvi, celu, bendis.")

array([0., 1., 0., 0., 0., 0.])

In [32]:
y_pred[:4]

[array([0., 1., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0.])]

In [33]:
multi_label_metrics(y_pred, y_true)

<ipython-input-20-0bd5108f3d62>:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  probs = sigmoid(torch.Tensor(predictions))


{'f1': 0.3083700440528634, 'roc_auc': 0.5, 'accuracy': 0.0}

In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_true=y_true, y_pred=y_pred, target_names=TARGET_LABELS))

              precision    recall  f1-score   support

           L       0.93      0.68      0.79        19
           G       0.96      0.98      0.97       133
           B       0.00      0.00      0.00         0
           T       0.75      0.90      0.82        10
           O       1.00      0.08      0.14        13
          NR       0.00      0.00      0.00         0

   micro avg       0.94      0.88      0.91       175
   macro avg       0.61      0.44      0.45       175
weighted avg       0.95      0.88      0.88       175
 samples avg       0.95      0.91      0.92       175



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
from huggingface_hub.hf_api import HfFolder

HfFolder.save_token('MY_TOKEN')

In [36]:
model.push_to_hub("homomex24-t2-beto-85-15")
tokenizer.push_to_hub("homomex24-t2-beto-85-15")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hiramochoavea/homomex24-t2-beto-85-15/commit/22ffcf863cb0f51904a17b4da4d71fcaaf944f0e', commit_message='Upload tokenizer', commit_description='', oid='22ffcf863cb0f51904a17b4da4d71fcaaf944f0e', pr_url=None, pr_revision=None, pr_num=None)